In [1]:
%pylab
from landlab import RasterModelGrid
from landlab.plot.imshow import imshow_grid_at_node
from landlab.components import PresFlowNetwork, MeltCreep
import numpy as np
from matplotlib import colors,  cm
import matplotlib.animation as animation


Using matplotlib backend: Qt5Agg
Populating the interactive namespace from numpy and matplotlib


In [5]:
mg = RasterModelGrid((20,30),100)
junc_elev = mg.add_zeros('node', 'junction__elevation')
R = mg.add_zeros('node', 'input__discharge')
#d_h = mg.add_ones('link', 'hydraulic__diameter')

d_h = mg.add_zeros('link','hydraulic__diameter')
mg.at_link['hydraulic__diameter'][mg.active_links]= 0.5*np.random.rand(mg.number_of_active_links)

h = mg.add_zeros('node', 'hydraulic__head')
thickness = 500.*np.ones(mg.number_of_nodes)
Z = mg.add_field('node', 'ice__thickness', thickness)
#a = mg.add_ones('link', 'head_loss__exponent')
#f = mg.add_ones('link', 'friction__factor')
#h = np.random.rand(mg.number_of_nodes)
Q = mg.add_zeros('link', 'conduit__discharge')
#net_node_flux = mg.add_ones('node', 'net_node_flux', noclobber=False)
#set boundary node head

#add input to node 310
moulin_Q=1.
mg.at_node['input__discharge'][310]=moulin_Q
#set heads at edges
h[mg.nodes_at_left_edge] = 0.#400.
h[mg.nodes_at_right_edge] = 300.
h[mg.nodes_at_top_edge] = 0.
h[mg.nodes_at_bottom_edge] = 0.
mg.set_closed_boundaries_at_grid_edges(False,True,True,True)
Q[mg.active_links] = 1.
n_core = mg.number_of_core_nodes
links = mg.links_at_node
print "Number of links = ", mg.number_of_links
print "Number of nodes = ", mg.number_of_nodes
print "Number of active links = ", mg.number_of_active_links
print "Number of core nodes = ", mg.number_of_core_nodes

pfn = PresFlowNetwork(mg)
pfn.run_one_step()
mc = MeltCreep(mg, dt=500.)

Number of links =  1150
Number of nodes =  600
Number of active links =  980
Number of core nodes =  504
Number of iterations = 1 tolerance = 1.64714366901
Number of iterations = 2 tolerance = 1.00502574121
Number of iterations = 3 tolerance = 0.984635441828
Number of iterations = 4 tolerance = 0.933725626936
Number of iterations = 5 tolerance = 0.806998072859
Number of iterations = 6 tolerance = 0.590761858376
Number of iterations = 7 tolerance = 0.33891886603
Number of iterations = 8 tolerance = 0.177740622799
Number of iterations = 9 tolerance = 0.0819273706845
Number of iterations = 10 tolerance = 0.0411126220079
Number of iterations = 11 tolerance = 0.0184539653788
Number of iterations = 12 tolerance = 0.00851794736602
Number of iterations = 13 tolerance = 0.00741064520776
Number of iterations = 14 tolerance = 0.00350673803571
Number of iterations = 15 tolerance = 0.00211757758542
Number of iterations = 16 tolerance = 0.000946386839351


In [3]:
figure()
imshow_grid_at_node(mg, h)

In [3]:
def plot_links(grid, value_name, autoscale=True, vmin=0., vmax=0., cmap_name='jet', magnitude= False, lw=2):
    link_head_x = grid.node_x[grid.node_at_link_head]
    link_head_y = grid.node_y[grid.node_at_link_head]
    link_tail_x = grid.node_x[grid.node_at_link_tail]
    link_tail_y = grid.node_y[grid.node_at_link_tail]
    if magnitude:
        values = abs(grid.at_link[value_name])
    else:
        values = grid.at_link[value_name]
    
    #Normalize color values
    if autoscale:
        cnorm = colors.Normalize()
        cnorm.autoscale(values)
    else:
        cnorm = colors.Normalize(vmin,vmax)
    scalarMap = cm.ScalarMappable(norm=cnorm, cmap = get_cmap(cmap_name))
    scalarMap.set_array(values)
    #set_cmap()
    for i, value in enumerate(values):
        xs = [link_head_x[i],link_tail_x[i]]
        ys = [link_head_y[i],link_tail_y[i]]
        img = plot(xs,ys,lw=lw,color=scalarMap.to_rgba(value))
    colorbar(scalarMap)


In [5]:
figure()
plot_links(mg, 'hydraulic__diameter', autoscale=False, vmin=0, vmax=1.)
print mean(mg.at_link['hydraulic__diameter'])

0.224942863279


In [6]:
figure()
plot_links(mg, 'conduit__discharge', autoscale=True, magnitude=True)
#print mean(mg.at_link['hydraulic__diameter'])

In [ ]:
nsteps = 5000
step_start = 5000
fig1, axs = subplots(1,3,figsize=(13,3))
every = 100
for step in arange(nsteps)+step_start:
    print "step =",step, " avg d_h=",mg.at_link['hydraulic__diameter'].mean()
    pfn.run_one_step()
    mc.run_one_step()
    if (step % every)==0: #make an animation frame
        subplot(1,3,1)
        imshow_grid_at_node(mg, h)
        subplot(1,3,2)
        plot_links(mg, 'conduit__discharge', vmin=0., vmax=1., magnitude=True, autoscale=False)
        subplot(1,3,3)
        plot_links(mg, 'hydraulic__diameter', autoscale=False, vmin=0, vmax=1.)
        image_name = 'heads_and_discharge'+str(step).zfill(6)+'.png'
        tight_layout()
        savefig(image_name)
        fig1.clf()

#im_ani = animation.ArtistAnimation(fig1,ims)

step = 5000  avg d_h= 0.0550196591746
Number of iterations = 1 tolerance = 0.000508942504042
step = 5001  avg d_h= 0.0549973458729
Number of iterations = 1 tolerance = 0.000508422209783
step = 5002  avg d_h= 0.0549750508559
Number of iterations = 1 tolerance = 0.000507901649485
step = 5003  avg d_h= 0.05495277411
Number of iterations = 1 tolerance = 0.000507380825407
step = 5004  avg d_h= 0.0549305156216
Number of iterations = 1 tolerance = 0.000506859739126
step = 5005  avg d_h= 0.0549082753772
Number of iterations = 1 tolerance = 0.000506338392852
step = 5006  avg d_h= 0.0548860533634
Number of iterations = 1 tolerance = 0.000505816788067
step = 5007  avg d_h= 0.0548638495666
Number of iterations = 1 tolerance = 0.000505294926826
step = 5008  avg d_h= 0.0548416639733
Number of iterations = 1 tolerance = 0.000504772811211
step = 5009  avg d_h= 0.05481949657
Number of iterations = 1 tolerance = 0.000504250442876
step = 5010  avg d_h= 0.0547973473433
Number of iterations = 1 tolerance =

Number of iterations = 1 tolerance = 0.000461808809453
step = 5090  avg d_h= 0.0530831629791
Number of iterations = 1 tolerance = 0.000461272377405
step = 5091  avg d_h= 0.0530624444723
Number of iterations = 1 tolerance = 0.000460735845649
step = 5092  avg d_h= 0.053041743163
Number of iterations = 1 tolerance = 0.000460199207163
step = 5093  avg d_h= 0.0530210590405
Number of iterations = 1 tolerance = 0.000459662468728
step = 5094  avg d_h= 0.0530003920942
Number of iterations = 1 tolerance = 0.000459125634025
step = 5095  avg d_h= 0.0529797423132
Number of iterations = 1 tolerance = 0.000458588706811
step = 5096  avg d_h= 0.0529591096868
Number of iterations = 1 tolerance = 0.000458051689922
step = 5097  avg d_h= 0.0529384942043
Number of iterations = 1 tolerance = 0.000457514585707
step = 5098  avg d_h= 0.052917895855
Number of iterations = 1 tolerance = 0.000456977397945
step = 5099  avg d_h= 0.0528973146282
Number of iterations = 1 tolerance = 0.000456440128039
step = 5100  avg 

Number of iterations = 1 tolerance = 0.000413362436452
step = 5180  avg d_h= 0.051286131858
Number of iterations = 1 tolerance = 0.000412824743572
step = 5181  avg d_h= 0.0512669190767
Number of iterations = 1 tolerance = 0.000412287118235
step = 5182  avg d_h= 0.0512477225594
Number of iterations = 1 tolerance = 0.000411749562081
step = 5183  avg d_h= 0.0512285422961
Number of iterations = 1 tolerance = 0.000411212076294
step = 5184  avg d_h= 0.0512093782766
Number of iterations = 1 tolerance = 0.000410674663953
step = 5185  avg d_h= 0.0511902304911
Number of iterations = 1 tolerance = 0.000410137325245
step = 5186  avg d_h= 0.0511710989295
Number of iterations = 1 tolerance = 0.000409600062866
step = 5187  avg d_h= 0.0511519835818
Number of iterations = 1 tolerance = 0.000409062878485
step = 5188  avg d_h= 0.0511328844379
Number of iterations = 1 tolerance = 0.000408525773333
step = 5189  avg d_h= 0.0511138014878
Number of iterations = 1 tolerance = 0.000407988749616
step = 5190  avg

Number of iterations = 1 tolerance = 0.000365435265404
step = 5270  avg d_h= 0.0496209542609
Number of iterations = 1 tolerance = 0.000364910355509
step = 5271  avg d_h= 0.0496031657038
Number of iterations = 1 tolerance = 0.000364385654314
step = 5272  avg d_h= 0.0495853925295
Number of iterations = 1 tolerance = 0.000363861163113
step = 5273  avg d_h= 0.0495676347285
Number of iterations = 1 tolerance = 0.000363336883467
step = 5274  avg d_h= 0.0495498922911
Number of iterations = 1 tolerance = 0.000362812816931
step = 5275  avg d_h= 0.0495321652079
Number of iterations = 1 tolerance = 0.000362288964647
step = 5276  avg d_h= 0.0495144534693
Number of iterations = 1 tolerance = 0.00036176532877
step = 5277  avg d_h= 0.0494967570656
Number of iterations = 1 tolerance = 0.000361241919603
step = 5278  avg d_h= 0.0494790759859
Number of iterations = 1 tolerance = 0.000360718853504
step = 5279  avg d_h= 0.049461410223
Number of iterations = 1 tolerance = 0.000360195787503
step = 5280  avg 

Number of iterations = 1 tolerance = 0.000319191630904
step = 5360  avg d_h= 0.0480804842588
Number of iterations = 1 tolerance = 0.000318690934028
step = 5361  avg d_h= 0.0480640425568
Number of iterations = 1 tolerance = 0.000318190562902
step = 5362  avg d_h= 0.0480476154003
Number of iterations = 1 tolerance = 0.000317690518707
step = 5363  avg d_h= 0.0480312027804
Number of iterations = 1 tolerance = 0.000317190802502
step = 5364  avg d_h= 0.0480148046879
Number of iterations = 1 tolerance = 0.000316691415641
step = 5365  avg d_h= 0.0479984211138
Number of iterations = 1 tolerance = 0.000316192358674
step = 5366  avg d_h= 0.0479820520489
Number of iterations = 1 tolerance = 0.000315693633354
step = 5367  avg d_h= 0.0479656974843
Number of iterations = 1 tolerance = 0.000315195240088
step = 5368  avg d_h= 0.0479493574109
Number of iterations = 1 tolerance = 0.000314697180675
step = 5369  avg d_h= 0.0479330318196
Number of iterations = 1 tolerance = 0.000314199455319
step = 5370  av

...

Number of iterations = 1 tolerance = 0.000198308735135
step = 5631  avg d_h= 0.0441284247738
Number of iterations = 1 tolerance = 0.000197925347708
step = 5632  avg d_h= 0.0441156109228
Number of iterations = 1 tolerance = 0.000197542455228
step = 5633  avg d_h= 0.0441028093444
Number of iterations = 1 tolerance = 0.000197160058279
step = 5634  avg d_h= 0.0440900200307
Number of iterations = 1 tolerance = 0.000196778156371
step = 5635  avg d_h= 0.0440772429738
Number of iterations = 1 tolerance = 0.000196396750202
step = 5636  avg d_h= 0.0440644781658
Number of iterations = 1 tolerance = 0.000196015839761
step = 5637  avg d_h= 0.0440517255988
Number of iterations = 1 tolerance = 0.000195635425276
step = 5638  avg d_h= 0.044038985265
Number of iterations = 1 tolerance = 0.000195255506737
step = 5639  avg d_h= 0.0440262571566
Number of iterations = 1 tolerance = 0.000194876085554
step = 5640  avg d_h= 0.0440135412657
Number of iterations = 1 tolerance = 0.000194497160316
step = 5641  avg

Number of iterations = 1 tolerance = 0.000165796752859
step = 5721  avg d_h= 0.0430234171944
Number of iterations = 1 tolerance = 0.000165458130263
step = 5722  avg d_h= 0.043011676829
Number of iterations = 1 tolerance = 0.000165119964751
step = 5723  avg d_h= 0.0429999480396
Number of iterations = 1 tolerance = 0.000164782186376
step = 5724  avg d_h= 0.0429882308175
Number of iterations = 1 tolerance = 0.000164444344213
step = 5725  avg d_h= 0.0429765251514
Number of iterations = 1 tolerance = 0.000164109211089
step = 5726  avg d_h= 0.0429648310401
Number of iterations = 1 tolerance = 0.000163772856366
step = 5727  avg d_h= 0.0429531484747
Number of iterations = 1 tolerance = 0.000163437404253
step = 5728  avg d_h= 0.0429414774481
Number of iterations = 1 tolerance = 0.00016310235409
step = 5729  avg d_h= 0.0429298179529
Number of iterations = 1 tolerance = 0.000162767801762
step = 5730  avg d_h= 0.0429181699815
Number of iterations = 1 tolerance = 0.000162433737646
step = 5731  avg 

Number of iterations = 1 tolerance = 0.000136713266164
step = 5813  avg d_h= 0.0419908161436
Number of iterations = 1 tolerance = 0.000136419778008
step = 5814  avg d_h= 0.0419801092914
Number of iterations = 1 tolerance = 0.000136126769547
step = 5815  avg d_h= 0.0419694133241
Number of iterations = 1 tolerance = 0.000135834241055
step = 5816  avg d_h= 0.0419587282344
Number of iterations = 1 tolerance = 0.000135542191363
step = 5817  avg d_h= 0.0419480540147
Number of iterations = 1 tolerance = 0.000135250620737
step = 5818  avg d_h= 0.0419373906576
Number of iterations = 1 tolerance = 0.00013495952921
step = 5819  avg d_h= 0.0419267381557
Number of iterations = 1 tolerance = 0.00013466891566
step = 5820  avg d_h= 0.0419160965015
Number of iterations = 1 tolerance = 0.0001343787806
step = 5821  avg d_h= 0.0419054656876
Number of iterations = 1 tolerance = 0.000134089122873
step = 5822  avg d_h= 0.0418948457066
Number of iterations = 1 tolerance = 0.000133799943517
step = 5823  avg d_

step = 5901  avg d_h= 0.0410894652011
Number of iterations = 1 tolerance = 0.000112436804543
step = 5902  avg d_h= 0.0410796877924
Number of iterations = 1 tolerance = 0.000112184769075
step = 5903  avg d_h= 0.0410699206159
Number of iterations = 1 tolerance = 0.00011193318277
step = 5904  avg d_h= 0.0410601636641
Number of iterations = 1 tolerance = 0.000111682044595
step = 5905  avg d_h= 0.0410504169298
Number of iterations = 1 tolerance = 0.000111431353961
step = 5906  avg d_h= 0.0410406804054
Number of iterations = 1 tolerance = 0.000111181112199
step = 5907  avg d_h= 0.0410309540838
Number of iterations = 1 tolerance = 0.000110931317997
step = 5908  avg d_h= 0.0410212379574
Number of iterations = 1 tolerance = 0.000110681972154
step = 5909  avg d_h= 0.041011532019
Number of iterations = 1 tolerance = 0.00011043307376
step = 5910  avg d_h= 0.0410018362612
Number of iterations = 1 tolerance = 0.000110184622217
step = 5911  avg d_h= 0.0409921506767
Number of iterations = 1 tolerance 

Number of iterations = 1 tolerance = 9.15038421971e-05
step = 5992  avg d_h= 0.0402407287017
Number of iterations = 1 tolerance = 9.12905372383e-05
step = 5993  avg d_h= 0.0402318523443
Number of iterations = 1 tolerance = 9.10776417143e-05
step = 5994  avg d_h= 0.0402229855518
Number of iterations = 1 tolerance = 9.08651555852e-05
step = 5995  avg d_h= 0.040214128317
Number of iterations = 1 tolerance = 9.06530781723e-05
step = 5996  avg d_h= 0.0402052806327
Number of iterations = 1 tolerance = 9.04414090335e-05
step = 5997  avg d_h= 0.0401964424915
Number of iterations = 1 tolerance = 9.02301474906e-05
step = 5998  avg d_h= 0.0401876138861
Number of iterations = 1 tolerance = 9.00192935084e-05
step = 5999  avg d_h= 0.0401787948093
Number of iterations = 1 tolerance = 8.98088460203e-05
step = 6000  avg d_h= 0.0401699852538
Number of iterations = 1 tolerance = 8.95988051541e-05


In [7]:
mc = MeltCreep(mg, dt=5000.)
#mean(mc.Q)

In [23]:
#figure()

pfn.run_one_step()
mc.run_one_step()

Number of iterations = 1 tolerance = 0.00299676026854
Number of iterations = 2 tolerance = 1.34229231785e-05
mean Q= 0.0179517502025


In [41]:
#for m in mc.melt:
#    print m
print max(mc.melt[~np.isnan(mc.melt)])
print max(mc.creep)
#print mc.d_h.mean()
#print mg.at_link['hydraulic__diameter']

3.35041761374e-06
2.12710591355e-06


In [15]:
#mc.run_one_step()

mc._grid.at_link['conduit__discharge']

array([ 0.,  0.,  0., ...,  0.,  0.,  0.])